In [27]:
puzzle = """
Four hikers (Ava, Ben, Ilya, Noor) each returned at a different time (6:10, 6:20, 6:30, 6:40 PM) to a different cabin (Cedar, Birch, Spruce, Aspen) carrying a different item (Compass, Journal, Lantern, Map). Determine who returned when, to which cabin, and with what.
Clues:
1) The person who returned at 6:20 went to Cedar, but did not carry the Lantern.
2) Noor didn’t go to Aspen and didn’t carry the Map.
3) Ava returned 10 minutes before the person who went to Birch.
4) The Compass carrier returned later than Ilya but earlier than the person headed to Spruce.
5) Ben went to Birch.
6) The Journal wasn’t taken to Cedar.
7) The person who went to Spruce returned 10 minutes after the Lantern carrier.
8) Ilya returned at 6:10 or 6:20.

Expected answer format:
6:10 — Person — Cabin — Item
6:20 — Person — Cabin — Item
6:30 — Person — Cabin — Item
6:40 — Person — Cabin — Item
"""

In [ ]:
from databricks_langchain import ChatDatabricks

llm = ChatDatabricks(
    model="databricks-gpt-oss-120b",
    extra_params={
        "reasoning_effort": "low"
    }
    )

response = llm.invoke(puzzle)

In [37]:
from pprint import pprint
pprint(response.content[0]['summary'][0]['text'])

("We need solve puzzle. Let's denote times T10=6:10, T20=6:20, T30=6:30, "
 'T40=6:40.\n'
 '\n'
 'Persons: Ava, Ben, Ilya, Noor.\n'
 '\n'
 'Cabins: Cedar, Birch, Spruce, Aspen.\n'
 '\n'
 'Items: Compass, Journal, Lantern, Map.\n'
 '\n'
 'Clue1: 6:20 person went to Cedar, no Lantern.\n'
 '\n'
 'Clue2: Noor not to Aspen, not Map.\n'
 '\n'
 'Clue3: Ava came 10min before Birch person. So if Birch at 6:10, Ava at 6:00 '
 "None. So Birch can't be earliest? Let's analyze: Ava 10min before Birch. So "
 'if Birch at 6:20, Ava at 6:10. If Birch at 6:30, Ava at 6:20. If Birch 6:40, '
 'Ava 6:30.\n'
 '\n'
 'Clue4: Compass carrier returned later than Ilya but earlier than Spruce '
 'person. So Ilya time < Compass time < Spruce time.\n'
 '\n'
 'Clue5: Ben went to Birch. So Birch cabin occupant is Ben.\n'
 '\n'
 'Clue6: Journal not to Cedar.\n'
 '\n'
 'Clue7: Spruce person returned 10 min after Lantern carrier. So Lantern time '
 '+10 = Spruce time.\n'
 '\n'
 'Clue8: Ilya 6:10 or 6:20.\n'
 '\n'
 'Now

In [46]:
from openai import OpenAI

from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
workspace_url = w.config.host
token = w.config.token

client = OpenAI(
    base_url=f"{workspace_url}/serving-endpoints",
    api_key=token
)

response = client.chat.completions.create(
    model="databricks-gpt-oss-120b",
    messages=[
        {"role": "user", "content": puzzle}
    ],
    reasoning_effort="low"
    )

In [47]:
pprint(response.choices[0].message.content[0]['summary'][0]['text'])

("We need solve. Let's brute think. Times: 10,20,30,40.\n"
 '\n'
 'Clue8: Ilya = 10 or 20.\n'
 '\n'
 'Clue1: 6:20 -> Cedar, not Lantern.\n'
 '\n'
 'So if Ilya 20 then Ilya at Cedar, not Lantern. If Ilya 10 then Cedar is '
 'someone else at 20.\n'
 '\n'
 'Clue5: Ben -> Birch.\n'
 '\n'
 'Clue3: Ava returned 10 min before person who went to Birch. So if Birch time '
 '= t, Ava time = t-10.\n'
 '\n'
 'Birch cabin is Ben. So time for Ben = t, Ava = t-10. Times possible: if '
 'Birch at 6:20, Ava 6:10. If Birch 6:30, Ava 6:20. If Birch 6:40, Ava 6:30. '
 "Can't be 6:10 because then Ava 6:00 not allowed. So possibilities: (Birch "
 '20, Ava 10) or (Birch 30, Ava 20) or (Birch 40, Ava 30).\n'
 '\n'
 'Consider each.\n'
 '\n'
 'Also clue7: Spruce time = Lantern carrier time +10.\n'
 '\n'
 'Clue4: Compass carrier time > Ilya time and < Spruce time.\n'
 '\n'
 'Clue2: Noor not Aspen, not Map.\n'
 '\n'
 'Clue6: Journal not Cedar.\n'
 '\n'
 "Let's attempt case1: Birch 20, Ava 10. Then Ben at 20 goes 